In [20]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import seaborn as sns
import os
import sys
import re
import string
import concurrent.futures
from tqdm import tqdm
from typing import List, Optional
from ignore_SSL_errors import no_ssl_verification


In [21]:
# Import list of euronext stocks => https://live.euronext.com/en/products/equities/list
# Important to remove rows 1 to 4 from csv file as they will create an error
file_name = "Euronext_Equities_2022-01-11.csv"
fields_list = ['Name', 'ISIN', 'Symbol', 'Market', 'Trading Currency']
stock_detail = pd.read_csv(file_name, sep=';', usecols=fields_list)
renaming_dict = {
    'Name': 'stock_name',
    'ISIN': 'stock_isin',
    'Symbol': 'stock_euronext_symbol',
    'Market': 'stock_exchange_markets',
    'Trading Currency': 'stock_trading_currency'
}
stock_detail.rename(columns=renaming_dict, inplace=True)
stock_detail.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR


In [22]:
# Get distinct stock details, we will need to try map this accordingly
stock_detail['stock_exchange_markets'].unique()

array(['Euronext Growth Paris', 'Oslo B�rs', 'Euronext Paris',
       'Euronext Expand Oslo', 'Euronext Access Paris',
       'Euronext Amsterdam', 'Euronext Growth Oslo', 'Euronext Brussels',
       'Euronext Expert Market', 'Euronext Brussels, Paris',
       'Euronext Paris, Amsterdam', 'Euronext Paris, Brussels',
       'Euronext Brussels, Amsterdam', 'Euronext Access Lisbon',
       'Euronext Amsterdam, Brussels', 'Euronext Dublin',
       'Euronext Lisbon', 'Euronext Amsterdam, Brussels, Paris',
       'Euronext Amsterdam, Paris', 'Traded not listed Brussels',
       'Euronext Growth Paris, Brussels', 'Euronext Growth Brussels',
       'Euronext Access Brussels', 'Euronext Growth Dublin',
       'Euronext Paris, Amsterdam, Brussels', 'Euronext Growth Lisbon'],
      dtype=object)

In [23]:
# Create a mapping for the exchange cities to yahoo codes
stock_exchange_markets_to_main_city_mapping = {
    'Amsterdam':'AS',
    'Brussels':'BR',
    'Dublin':'IR',
    'Lisbon':'LS',
    'Oslo':'OL',
    'Paris':'PA' 
    }


In [24]:
# Create a unique list of cities
cities_list = list(stock_exchange_markets_to_main_city_mapping.keys())
cities_list


['Amsterdam', 'Brussels', 'Dublin', 'Lisbon', 'Oslo', 'Paris']

In [25]:
# Function that takes in a string (input_text) and a list of different cities
# Returns the first city mentioned in the string that is in the cities list
def return_first_city(input_text: str, cities_list: List[str]) -> str:
    split_word_list = re.sub('['+string.punctuation+']', '', input_text).split()
    for word in split_word_list:
      if word in cities_list:
        result = word
        break
      else:
        result = "NA"
    return result


# Quick test to see required logic works
assert return_first_city('Amsterdam, Paris, Brussels', cities_list) == 'Amsterdam', 'Check logic'
assert return_first_city('Brussels, Amsterdam, Paris, Brussels',
                         cities_list) == 'Brussels', 'Check logic'


In [26]:
# Create column to get the main city
stock_detail['main_city'] = stock_detail['stock_exchange_markets'].apply(
    lambda x: return_first_city(x, cities_list))
stock_detail.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,Paris
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK,Oslo
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,Paris
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,Paris
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,Paris


In [27]:
# Get the Yahoo city mapping
stock_detail['yahoo_city_code'] = stock_detail['main_city'].map(
    stock_exchange_markets_to_main_city_mapping)
stock_detail.head()


,stock_name,stock_isin,stock_euronext_symbol,stock_exchange_markets,stock_trading_currency,main_city,yahoo_city_code
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,Paris,PA
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK,Oslo,OL
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,Paris,PA
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,Paris,PA
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,Paris,PA


In [28]:
# Create the full city code
stock_detail['yahoo_code'] = stock_detail['stock_euronext_symbol'].str.cat(
    stock_detail['yahoo_city_code'], sep='.')


In [29]:
# Filter to get the fields needed and save to a csv
stock_detail.to_csv('data_stock_info.csv', sep='|', index=False)

In [30]:
# https://stackoverflow.com/questions/8391411/how-to-block-calls-to-print
# Function to prevent excess printing from yfinance
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


In [31]:
# Create a function that extracts price data from yahoo using a ticker code
def get_pricing_data(yahoo_code: str) -> Optional[pd.DataFrame]:
    price_history = None
    try:
        with HiddenPrints():
            ticker = yf.Ticker(yahoo_code)
            price_history = ticker.history(period="5y")
            # If the result is a data frame, add come additional fields and add it to a list
            if isinstance(price_history, pd.DataFrame):
                renaming_dict = {
                    'Date': 'date',
                    'Open': 'open_price',
                    'High': 'high_price',
                    'Low': 'low_price',
                    'Close': 'close_price',
                    'Adj Close': 'adjusted_close_price',
                    'Volume': 'trading_volume',
                }
                price_history.reset_index(inplace=True)
                price_history.rename(columns=renaming_dict, inplace=True)
    except:
        price_history = None
    return price_history

# Test
with no_ssl_verification():
    apple_returns = get_pricing_data('AAPL')
apple_returns.head()


,date,open_price,high_price,low_price,close_price,trading_volume,Dividends,Stock Splits
0,2017-01-17,27.930251,28.378684,27.901931,28.322041,137759200,0.0,0.0
1,2017-01-18,28.322038,28.440047,28.253593,28.319677,94852000,0.0,0.0
2,2017-01-19,28.180433,28.343283,28.173353,28.270119,102389200,0.0,0.0
3,2017-01-20,28.428247,28.428247,28.258317,28.322041,130391600,0.0,0.0
4,2017-01-23,28.322042,28.513215,28.267757,28.340923,88200800,0.0,0.0


In [32]:
# Define function to run through all the ticker codes and extract price data
def iterate_tickers(tickers = List[str])-> List[pd.DataFrame]:
    results_list = list()
    with tqdm(total=len(tickers)) as progress_bar:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(
                get_pricing_data, yahoo_code): yahoo_code for yahoo_code in tickers}
            for future in concurrent.futures.as_completed(futures):
                df = future.result()
                if df is not None:
                    results_list.append(df)
                progress_bar.update(1)
    return results_list


In [34]:
tickers = stock_detail['yahoo_code'].to_list()
with no_ssl_verification():
  final_price_history_list = iterate_tickers(tickers)


100%|██████████| 1790/1790 [03:07<00:00,  9.56it/s]


In [ ]:
# Combine all the final results and save to csv
final_price_history_data = pd.concat(final_price_history_list)
final_price_history_data.to_csv('data_price_history.csv', sep='|', index=False)